In [3]:
import os
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from PIL import Image
import glob
import cv2
from sklearn.model_selection import train_test_split
from keras.layers import Dropout, Dense
from keras.layers.normalization import BatchNormalization
from keras.models import Sequential, load_model
from keras.applications import VGG16
from sklearn.metrics import accuracy_score, confusion_matrix

Using TensorFlow backend.


In [4]:
def model(input_shape):
    model = Sequential()
        
    model.add(Dense(512, activation='relu', input_dim = input_shape))
    model.add(Dropout(0.1))
    
    model.add(Dense(256, activation='relu'))
    
    model.add(Dense(128, activation='relu'))
    model.add(BatchNormalization())
    
    model.add(Dense(64, activation='relu'))
    model.add(Dense(output_dim = no_of_classes, activation='softmax')) 
    
    return model

In [5]:
from keras.models import load_model
no_of_classes = 7
model_top = model(10 * 10 * 512)
EMOTION_DICT = {1: "ANGRY", 2: "DISGUST", 3: "FEAR", 4: "HAPPY", 5: "NEUTRAL", 6: "SAD", 7: "SURPRISE"}
model_VGG = VGG16(weights='imagenet', include_top=False)
model_top.load_weights('model_weights.h5')

Instructions for updating:
Colocations handled automatically by placer.


C:\Users\xinrui zhan\Anaconda3\lib\site-packages\ipykernel_launcher.py:13: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(activation="softmax", units=7)`
  del sys.path[0]


58892288/58889256 [==============================] - 47s 1us/step


In [6]:
model_top = load_model("model_weights.h5")

ValueError: Cannot create group in read-only mode.

In [ ]:
def make_prediction(path):
    #converting image to gray scale and save it
    img = cv2.imread(path)
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    cv2.imwrite(path, gray)
    print('enter')
    #detect face in image, crop it then resize it then save it
    face_cascade = cv2.CascadeClassifier('haarcascade_frontalface_default.xml') 
    img = cv2.imread(path)
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    faces = face_cascade.detectMultiScale(gray, 1.3, 8)
    for (x,y,w,h) in faces:
        face_clip = img[y:y+h, x:x+w]
        new_face_path = path.split('/')
        new_face_path = new_face_path[len(new_face_path)-1]
        new_face_path = new_face_path.rsplit('.',1)
        new_face_path = new_face_path[0] + '-FACE.jpg'
        print(new_face_path)
        if not os.path.exists('faces'):
            os.makedirs('faces')
        cv2.imwrite('faces/'+new_face_path, cv2.resize(face_clip, (350, 350)))
    if faces == ():
        return "No faces detected!"
    #read the processed image then make prediction and display the result
    read_image = cv2.imread('faces/'+new_face_path)
    read_image = read_image.reshape(1, read_image.shape[0], read_image.shape[1], read_image.shape[2])
    read_image_final = read_image/255.0  #normalizing the image
    VGG_Pred = model_VGG.predict(read_image_final)  #creating bottleneck features of image using VGG-16.
    VGG_Pred = VGG_Pred.reshape(1, VGG_Pred.shape[1]*VGG_Pred.shape[2]*VGG_Pred.shape[3])
    top_pred = model_top.predict(VGG_Pred)  #making prediction from our own model.
    emotion_label = top_pred[0].argmax() + 1
    print("Predicted Expression Probabilities")
    print("ANGRY: {}\nDISGUST: {}\nFEAR: {}\nHAPPY: {}\nNEUTRAL: {}\nSAD: {}\nSURPRISE: {}\n\n".format(top_pred[0][0], top_pred[0][1], top_pred[0][2], top_pred[0][3], top_pred[0][4], top_pred[0][5], top_pred[0][6]))
    os.rename('faces/'+new_face_path, 'faces/' + new_face_path.rsplit('.',1)[0] + '-' + str(EMOTION_DICT[emotion_label]) +'.jpg')
    print("Dominant Probability = "+str(EMOTION_DICT[emotion_label])+": "+str(max(top_pred[0])))

In [ ]:
import glob
import os

files = glob.glob("../dsc160-midterm-group13/data/scenes/incredibles_example/*.jpg")
for fname in files:
        make_prediction(fname)
        print(fname)